In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-01-09 20:38:30--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  2.68MB/s    in 0.3s    

2023-01-09 20:38:31 (2.68 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Project-4").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from config import password

In [5]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://amazonsampledata.s3.amazonaws.com/final_genre_12k.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("final_genre_12k.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
df.show()

+---+--------------------+--------------------+-------------------+--------------------+-------------+--------------+-------+------+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|_c0|            track_id|               track|             artist|               genre|primary_genre|   duration_ms|    key|  mode|time_signature|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|  tempo|
+---+--------------------+--------------------+-------------------+--------------------+-------------+--------------+-------+------+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|  0|3e9HZxeyfWwjeyPAM...|        thank-u-next|      Ariana-Grande|['R&B', 'Pop', 'E...|          R&B|        207320|      1|     1|             4|       0.229|       0.717| 0.653|             0.0|   0.101|  -5.634|     0.0658|  0.412|106.966|
|  1|5p7ujcrUXASCNwRaW..

In [6]:
#Create necessary df
df.count()

11714

In [7]:
len(df.columns)

19

In [8]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
pop_df = df.filter(col("primary_genre")  == 'Pop')
pop_df.show()

+---+--------------------+--------------------+--------------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|_c0|            track_id|               track|              artist|               genre|primary_genre|duration_ms|key|mode|time_signature|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|  tempo|
+---+--------------------+--------------------+--------------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|  4|0MMSmg7zyo6pOKZrf...|A-Holly-Jolly-Chr...|           Burl-Ives|['Pop', 'Holiday'...|          Pop|     133547|  0|   1|             4|       0.685|       0.665| 0.398|             0.0|   0.172| -11.886|       0.03|  0.864|140.456|
|  6|1xzBco0xcoJEDXktl...|            Mo-Bamba|         

In [9]:
pop_df.count()

4975

In [20]:
pop_df = pop_df.drop('track_id','genre','duration_ms')

In [ ]:
pop_df.show()

In [10]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
rock_df = df.filter(col("primary_genre")  == 'Rock')
rock_df.show()

+---+--------------------+--------------------+-----------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|_c0|            track_id|               track|           artist|               genre|primary_genre|duration_ms|key|mode|time_signature|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|  tempo|
+---+--------------------+--------------------+-----------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|  5|6Z924AupOiJLdnAKH...|    Jingle-Bell-Rock|      Bobby-Helms|['Rock', 'Country...|         Rock|     130973|  2|   1|             4|       0.651|       0.757| 0.428|             0.0|  0.0644|  -8.537|      0.036|  0.816|119.813|
| 18|2FY7b99s15jUprqC0...|             Natural|  Imagine-Dragons|['R

In [11]:
rock_df.count()

2771

In [22]:
rock_df = rock_df.drop('track_id','genre','duration_ms')

In [12]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
rb_df = df.filter(col("primary_genre")  == 'R&B')
rb_df.show()

+---+--------------------+--------------------+---------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|_c0|            track_id|               track|         artist|               genre|primary_genre|duration_ms|key|mode|time_signature|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|  tempo|
+---+--------------------+--------------------+---------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|  0|3e9HZxeyfWwjeyPAM...|        thank-u-next|  Ariana-Grande|['R&B', 'Pop', 'E...|          R&B|     207320|  1|   1|             4|       0.229|       0.717| 0.653|             0.0|   0.101|  -5.634|     0.0658|  0.412|106.966|
|  1|5p7ujcrUXASCNwRaW...|          Without-Me|         Halsey|['R&B', 'Pop'

In [13]:
rb_df.count()

1863

In [23]:
rb_df = rb_df.drop('track_id','genre','duration_ms')

In [14]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
country_df = df.filter(col("primary_genre")  == 'Country')
country_df.show()

+---+--------------------+--------------------+--------------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|_c0|            track_id|               track|              artist|               genre|primary_genre|duration_ms|key|mode|time_signature|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|  tempo|
+---+--------------------+--------------------+--------------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
| 23|698eQRku24PIYPQPH...|She-Got-the-Best-...|          Luke-Combs|['Country', 'Pop ...|      Country|     183160| 11|   1|             4|      0.0292|       0.533| 0.907|             0.0|   0.386|  -3.793|     0.0406|    0.7| 150.99|
| 24|1G4cdOtF36ZCs6i2d...|            Drunk-Me|   Mitche

In [15]:
country_df.count()

1274

In [24]:
country_df = country_df.drop('track_id','genre','duration_ms')

In [16]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
non_music_df = df.filter(col("primary_genre")  == 'Non-Music')
non_music_df.show()

+----+--------------------+--------------------+----------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
| _c0|            track_id|               track|          artist|               genre|primary_genre|duration_ms|key|mode|time_signature|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|  tempo|
+----+--------------------+--------------------+----------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|2531|7lXbmBVZsyhxsxBzo...|     Looking-for-You|   Kirk-Franklin|['Non-Music', 'Re...|    Non-Music|     246560|  1|   1|             4|      0.0138|        0.81|  0.72|             0.0|   0.221|  -4.793|     0.0503|  0.805|121.984|
|3958|03L32Cqbq5lKcTg6C...|             Passion|              K5|['N

In [17]:
non_music_df.count()

9

In [18]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
rap_df = df.filter(col("primary_genre")  == 'Rap')
rap_df.show()

+---+--------------------+-------------------+--------------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|_c0|            track_id|              track|              artist|               genre|primary_genre|duration_ms|key|mode|time_signature|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|  tempo|
+---+--------------------+-------------------+--------------------+--------------------+-------------+-----------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|  2|2xLMifQCjDGFmkHkp...|         SICKO-MODE|        Travis-Scott|['Rap', 'Memes', ...|          Rap|     312820|  8|   1|             4|     0.00513|       0.834|  0.73|             0.0|   0.124|  -3.714|      0.222|  0.446|155.008|
|  9|7dt6x5M1jzdTEt8oC...|         Better-Now|         Post-

In [19]:
rap_df.count()

816

In [26]:
rap_df = rap_df.drop('track_id','genre','duration_ms')

In [27]:
final_df = pop_df.union(rock_df).union(rb_df).union(country_df).union(rap_df)


In [30]:
final_df.show()

+---+--------------------+--------------------+-------------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|_c0|               track|              artist|primary_genre|key|mode|time_signature|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|  tempo|
+---+--------------------+--------------------+-------------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|  4|A-Holly-Jolly-Chr...|           Burl-Ives|          Pop|  0|   1|             4|       0.685|       0.665| 0.398|             0.0|   0.172| -11.886|       0.03|  0.864|140.456|
|  6|            Mo-Bamba|           Sheck-Wes|          Pop|  4|   1|             4|       0.194|       0.729| 0.625|         0.00986|   0.248|  -5.266|     0.0315|  0.261|146.034|
|  8|          Youngblood| 5-Seconds-of-Summer|          Pop|  7|   0|             4|     

In [31]:
from pyspark.sql.functions import monotonically_increasing_id
final_df = final_df.withColumn("index", monotonically_increasing_id())

In [32]:
final_df.show()

+---+--------------------+--------------------+-------------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+-----+
|_c0|               track|              artist|primary_genre|key|mode|time_signature|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|  tempo|index|
+---+--------------------+--------------------+-------------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+-----+
|  4|A-Holly-Jolly-Chr...|           Burl-Ives|          Pop|  0|   1|             4|       0.685|       0.665| 0.398|             0.0|   0.172| -11.886|       0.03|  0.864|140.456|    0|
|  6|            Mo-Bamba|           Sheck-Wes|          Pop|  4|   1|             4|       0.194|       0.729| 0.625|         0.00986|   0.248|  -5.266|     0.0315|  0.261|146.034|    1|
|  8|          Youngblood| 5-Seconds-of-Summer|          Pop

In [33]:
final_df = final_df.drop('_c0')

In [34]:
final_df.show()

+--------------------+--------------------+-------------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+-----+
|               track|              artist|primary_genre|key|mode|time_signature|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|  tempo|index|
+--------------------+--------------------+-------------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+-----+
|A-Holly-Jolly-Chr...|           Burl-Ives|          Pop|  0|   1|             4|       0.685|       0.665| 0.398|             0.0|   0.172| -11.886|       0.03|  0.864|140.456|    0|
|            Mo-Bamba|           Sheck-Wes|          Pop|  4|   1|             4|       0.194|       0.729| 0.625|         0.00986|   0.248|  -5.266|     0.0315|  0.261|146.034|    1|
|          Youngblood| 5-Seconds-of-Summer|          Pop|  7|   0|             4

In [35]:
# Get the list of columns in the DataFrame
columns = final_df.columns

# Reorder the columns by moving the last column to the first position
columns = columns[-1:] + columns[:-1]

# Select the columns in the specified order
final_df = final_df.select(*columns)


In [36]:
final_df.show()

+-----+--------------------+--------------------+-------------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|index|               track|              artist|primary_genre|key|mode|time_signature|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|  tempo|
+-----+--------------------+--------------------+-------------+---+----+--------------+------------+------------+------+----------------+--------+--------+-----------+-------+-------+
|    0|A-Holly-Jolly-Chr...|           Burl-Ives|          Pop|  0|   1|             4|       0.685|       0.665| 0.398|             0.0|   0.172| -11.886|       0.03|  0.864|140.456|
|    1|            Mo-Bamba|           Sheck-Wes|          Pop|  4|   1|             4|       0.194|       0.729| 0.625|         0.00986|   0.248|  -5.266|     0.0315|  0.261|146.034|
|    2|          Youngblood| 5-Seconds-of-Summer|          Pop|  7|   0|        

In [37]:
final_df.write.option("sep", ",").option("header", "true").csv("/content/csv/equalized_data.csv")


# **Load**

In [ ]:
mode = "overwrite"
jdbc_url="jdbc:postgresql://big-data-base.coodiqlcstgq.us-east-1.rds.amazonaws.com:5432/final_project_db"
config = {"user":"postgres", "password": password, "driver":"org.postgresql.Driver"}

In [ ]:
# Write final_df to table in RDS
final_df.write.jdbc(url=jdbc_url, table='final_df', mode=mode, properties=config)